# **Presentación de resultados**

## **Suposiciones inciales**

El problema se enfocó en la clasificación de latidos para la detección de arritmias cardiacas, a partir de una imagen de electrocardiograma de 12 derivaciones o un CSV.


Para ello, se tomó como base el planteamiento ofrecido en Kaggle bajo el título:

**ECG Heartbeat Categorization Dataset - Segmented and Preprocessed ECG Signals for Heartbeat Classification**
el cual está disponible en: https://www.kaggle.com/datasets/shayanfazeli/heartbeat

Para este problema, se ofrece un dataset preprocesado con las siguientes caracterticas:

    Arrhythmia Dataset

    Number of Samples: 109446
    Number of Categories: 5
    Sampling Frequency: 125Hz
    Data Source: Physionet's MIT-BIH Arrhythmia Dataset
    Classes: ['N': 0, 'S': 1, 'V': 2, 'F': 3, 'Q': 4]

Este dataset ofrece segmentos de latidos con la siguiente forma:

![imagen.png](https://drive.google.com/uc?export=view&id=1VwTHkjTI4s-2T5r-S-NToQmGnOFvZogp)

Con lo anterior, se asumió que este dataset de 109,446 latidos con 187 características cada uno sería suficiente para avanzar en la construcción de la arquitectura de la red neuronal y posteriormente, clasificar el latido que se extrae de la imagen.

#### **Problema**

El principal problema se tuvo al momento de segmentar el latido del vector que se extrajo de la imagen:

![imagen.png](https://drive.google.com/uc?export=view&id=1GwLSDZQETBoT8vXeICkmZEcid9kfx8B3)

Las variaciones en el inicio del segmento, se traducian en resultados distintos en el momento de clasificarle con el modelo entrenado.

![imagen.png](https://drive.google.com/uc?export=view&id=1AlqUCCmLXwPJ1dSdlBXoZeBA_2cDUD9s)


Derivado de lo anterior, se optó por preparar un nuevo dataset a partir de la base de datos original disponible directamente en Physionet con el nombre MIT-BIH Arrhythmia Database.

La decisión se sustentó en implementar una estrategia de segmentación de latidos en la cual tuviéramos control de ello y se pudiera replicar sin problema en el preprocesamiento de la imagen y CSV que se enviaría a analizar con el modelo desde la aplicación móvil.

Esto asumiendo que el no incluir ese fragmento de la onda R al inicio de la segmención, ayudaría en la robuztes de la clasificación.

**Validación**

El mecanismo de validación, fue cuidar que para cada segmento de latido se pudieran incluir las ondas que le caracterizan.

![imagen.png](https://drive.google.com/uc?export=view&id=1svSrqPuj3pen7jd-O1Vj0EVxAamSFCeh)



## **Métricas seleccionadas**

En el proceso de segmentación de latidos, las acciones relevantes que se aplicaron son:
    
    - Segmentar en bloques de 10 segundos la información
    - Normalizar valores entre 0 y 1 los 10 segundos y sobre ello implementar el procesamiento
    - Determinar la ventana de extracción del latido
    - Ajustar la frecuencia de muestreo del vector a 125Hz
    
¿Por qué ajustar la frecuencia de muestreo a 125Hz?
    
    - Eficiencia computacional: Al disminuir la frecuencia de muestreo, el volumen de datos a procesar y almacenar se reduce significativamente. Para expertos en aprendizaje profundo y big data, esta reducción puede ser crucial para acelerar el tiempo de entrenamiento y limitar los requisitos de almacenamiento, especialmente cuando se trabaja con grandes volúmenes de datos y modelos complejos como las CNN.
    
    - Filtrado de ruido y mejoramiento de señal: Al reducir la frecuencia de muestreo, se eliminan las componentes de alta frecuencia, lo que también puede eliminar el ruido no deseado en las señales de ECG, como artefactos musculares y otros factores no cardíacos. Esto facilita la identificación y extracción de características relevantes del ECG por parte de las CNN, lo que puede mejorar la precisión del modelo y su capacidad para generalizar.
    
    - Reducción de dimensionalidad y prevención del sobreajuste: La disminución de la frecuencia de muestreo conlleva una reducción en la dimensionalidad de los datos. Esto simplifica los modelos de CNN y disminuye el riesgo de sobreajuste, lo que es especialmente relevante en contextos de aprendizaje profundo donde se manejan grandes cantidades de parámetros. Además, la reducción de dimensionalidad facilita la visualización y el análisis de los datos, lo que puede ser útil para comprender mejor el rendimiento del modelo y la importancia de las características extraídas.
    
Considerando lo anterior, se determinó utilizar una ventana de segmentación para cada bloque de 10 segundos, igual a la duración máxima del latido en ese periodo. 

Esto nos arrojó vectores de longitud desde unas 70 muestras, hasta poco más de 175 muestras. Razón por la cual se replicó lo que en varios artículos indcaban, utilizar una longitud de 187, rellenando con ceros los valores faltantes.

Esto nos llevó a vigilar las siguientes métricas en el proceso de entrenamiento:

![imagen.png](https://drive.google.com/uc?export=view&id=1wlh17oUkwC2FfmhIsC7AKotVC68iYPeP)

    - Precision. La precisión mide la proporción de verdaderos positivos (latidos correctamente clasificados) entre todos los casos identificados como positivos por el modelo. En el contexto de la clasificación de latidos de ECG, la precisión es importante porque indica qué tan confiable es el modelo al identificar latidos de cierta clase.
    
    - Recall (sensibilidad): El recall mide la proporción de verdaderos positivos (latidos correctamente clasificados) entre todos los casos reales positivos en el dataset. En el contexto de la clasificación de latidos de ECG, el recall es importante porque indica qué tan bien el modelo identifica todos los latidos relevantes de una clase específica, lo cual es especialmente crítico cuando se trata de detectar latidos anormales que podrían indicar problemas cardíacos.
    
    - 
    


**¿Fue correcta la selección?**

Definitivamente el resultado fue satisfactorio al evaluar la segmentación de los latidos extraidos de la imagen, ya que se logró obtener una buena clasificación.


## **Arquitectura**

Con relación a la arquitectura:

![imagen.png](https://drive.google.com/uc?export=view&id=1teqRctm0i8rdk1k0WU1JZLw45iTNX11i)


    - Capa de convolución 1D (Conv1D): Esta capa utiliza filtros convolucionales 1D para extraer características de los datos de entrada. Tiene 128 filtros y un tamaño de kernel de 3. Los pesos en esta capa se inicializan con 640 parámetros.

    - Capa de activación (ReLU): Aplica la función de activación ReLU (Rectified Linear Unit) a las salidas de la capa de convolución, lo que introduce la no linealidad en el modelo.

    - Capa de normalización por lotes (BatchNormalization): Esta capa normaliza las salidas de la capa de activación, lo que ayuda a acelerar el entrenamiento y mejora la estabilidad del modelo.

    - Capa de MaxPooling1D: Realiza la operación de max-pooling con un tamaño de ventana de 2, lo que reduce las dimensiones temporales de los datos y permite que el modelo se enfoque en las características más relevantes.

    - Capa de Dropout: Aplica un dropout del 50% a las salidas de la capa de MaxPooling1D, lo que ayuda a prevenir el sobreajuste durante el entrenamiento.

    - Capa de convolución 1D (Conv1D): Similar a la primera capa de convolución, pero con 256 filtros y 131,328 parámetros.

    - Capa de activación (ReLU): Aplica la función de activación ReLU a las salidas de la segunda capa de convolución.

    - Capa de normalización por lotes (BatchNormalization): Normaliza las salidas de la segunda capa de activación.

    - Capa de MaxPooling1D: Realiza max-pooling con un tamaño de ventana de 2 en las salidas de la segunda capa de normalización por lotes.

    - Capa de Dropout: Aplica un dropout del 50% a las salidas de la segunda capa de MaxPooling1D.

    - Capa de aplanado (Flatten): Aplana las salidas de la segunda capa de Dropout, convirtiendo los datos en un vector unidimensional.

    - Capa densa (Dense): Una capa densa (fully connected) con 416 neuronas y función de activación ReLU. Tiene 4,792,736 parámetros.

    - Capa de Dropout: Aplica un dropout del 50% a las salidas de la primera capa densa.

    - Capa densa (Dense): Una segunda capa densa con 128 neuronas y función de activación ReLU. Tiene 53,376 parámetros.

    - Capa de Dropout: Aplica un dropout del 50% a las salidas de la segunda capa densa.

    - Capa densa (Dense): Una tercera capa densa con 64 neuronas y función de activación ReLU. Tiene 8,256 parámetros.

    - Capa de Dropout: Aplica un dropout del 50% a las salidas de la tercera capa densa.

    - Capa de salida (Dense): La capa de salida tiene 5 neuronas, una para cada clase de latido de ECG. Utiliza la función de activación softmax para producir probabilidades de pertenencia a cada clase. Tiene 325 parámetros.

Esta arquitectura fue el resultado de la evolución a partir de una inicial que solo incluia capas densas. 

**¿Ha sido la correcta?**

Sus resultados mostrados en el punto anterior y durante las pruebas, han sido satisfactorios en cuanto a precisión en la clasificación.



## **Métodos ML utilizados**

Se comenzó con un modelo de red neuronal integrada solo por capas densas. Si bien clasificaba latidos, no lograba generalizar el resultado con el conjunto de datos de prueba.

Revisando artículos, se encontraron diversas estrategias para abordar la clasificación de latidos. Entre ellos fue el uso de redes neuronales con capas convolucionales 1D.

Los artículos ofrecen resultados más acertivos en la clasificación con arquitecturas más complejas, sin embargo, este modelo propuesto y con el entrenamiento realizado en un equipo local con una GPU, proporcionó resultados satisfactorios en cuanto a precisión en la clasificación.




### **Aprendizaje y mejoras**

**Sobre los dataset**

Una de las lecciones aprendidas es poner más atención en los datasets utilizados para entrenar modelos. Si bien existen conjuntos de datos que pueden simplificar o incluso, omitir la etapa de exploración y preprocesamiento para obtener el dataset, esto puede convertirse en una doble dedicación de tiempo y esfuerzo si no se tiene comprensión del tratamiento realizado.

En este caso, la solución fue regresar a los datos originales y procesar un dataset con el control total de los criterios de segmentación.


**Estado del arte**

Este proyecto se empujó a desarrollar una solución desde cero con las herramientas proporcionadas en el bootcamp. En una próxima solución, se revisará el estado del arte sobre soluciones similares y comenzar a partir de lo que ya esté hecho. Por ejemplo, buscar modelos ya entrenados y ajustarlos a la necesidad específica mediante transfer learning.

**Dominio del problema**

Queda evidente la relevancia del dominio que se debe tener en el fenómeno que representan los datos. Esto ayudó para poder abordar desde cero la generación del dataset para entrenamiento. En caso contrario, al no tener claridad del fenómeno que representan los datos, se estaría dispersando esfuerzo, tiempo y recursos en acciones que no sumarían a lograr la solución.

**Nuevo dataset**

Si bien el modelo desarrolloado fue evaluado su propio conjunto de datos de prueba de ECG parteneciente a la misma base de datos:

    - Entrenamiento: MIT-BIH Arrhythmia Database (https://physionet.org/content/mitdb/1.0.0/)

Se hizo el ejercicio de evaluar con muestras de una nueva base de datos:

    - Nueva base de datos: Icentia11k Single Lead Continuous Raw Electrocardiogram Dataset (https://physionet.org/content/icentia11k-continuous-ecg/1.0/)
    
¿Qué características tiene este nuevo dataset publicado en abril de 2022?

    - Número de pacientes: 11,000
    - Número de etiquetas en latidos: 2,774,054,987
    - Frecuencia de muestreo: 250Hz
    - Tamaño de base de datos: 1.1 TB
    
Se propone seguir evaluando el modelo actual con esta nueva base de datos y trabajar en extender el entrenamiento del modelo, para mejorar sus capacidades de generalizar la clasificación, al disponer de más de 2 mil millones de latidos etiquetados.



## **Información del dataset**

Se utilizó la base de datos en formato WFDB de Physionet indentificada como "MIT-BIH Arrhythmia Database" disponible en https://www.physionet.org/content/mitdb/1.0.0/

¿Qué es el formato WFDB?

El formato WFDB (WaveForm DataBase) es un conjunto de formatos de archivo y bibliotecas de software desarrolladas en el MIT (Instituto Tecnológico de Massachusetts) para el almacenamiento y manipulación de señales y anotaciones utilizadas en la investigación biomédica, especialmente en el análisis de señales de electrocardiogramas (ECG) y otras señales fisiológicas.

El formato WFDB es ampliamente utilizado en la comunidad de investigadores de ECG, y las bases de datos de ECG, como la base de datos MIT-BIH Arrhythmia Database, se distribuyen en este formato.

Los archivos WFDB constan de dos componentes principales:

    1. Archivos de señal: Contienen los valores numéricos de las señales muestreadas, que pueden ser multiplexados (múltiples señales almacenadas juntas) o no. Estos archivos suelen tener extensiones como ".dat" o ".bin".

    2. Archivos de encabezado: Contienen metadatos sobre las señales, como la frecuencia de muestreo, la resolución, el número de señales, la duración de la grabación y otros detalles relevantes. Los archivos de encabezado suelen tener la extensión ".hea".
    

**¿Qué información hay disponible en la base de datos?**

Esta información tiene las siguientes características generales:

    48 extractos de media hora de registro de ECG
    La información pertenece a 47 personas diferentes
    Las grabaciones contienen trazos con una frecuencia de muestreo de 360Hz
    Aproximadamente 110,000 anotaciones totales
    
Las más de 110 mil anotaciones se encuentran en el estardar de MIT-BIH, que son 15 clases:

![imagen.png](https://drive.google.com/uc?export=view&id=13ZmZhK3_Ww5AFtWn_eujbFSSRW034sv7)


Estas clases se agruparon en las 5 que establece el estandar AAMI.

¿Qué es la clasificación AAMI?

La clasificación estándar AAMI (Association for the Advancement of Medical Instrumentation) en latidos de ECG es un conjunto de directrices que definen un esquema de etiquetado para diferentes tipos de latidos cardíacos en registros de electrocardiograma (ECG). Esta clasificación estandarizada ayuda a la comunidad médica y científica a comparar y compartir resultados de investigación y análisis de latidos de ECG de manera coherente y uniforme.

La clasificación AAMI divide los latidos de ECG en cinco clases principales, que se etiquetan como N, V, S, F y Q. Estas etiquetas representan lo siguiente:

    N: Latidos normales o supraventriculares (también conocidos como latidos sinusal). Estos latidos se originan en el nódulo sinusal y representan el ritmo cardíaco normal y saludable.

    V: Latidos ventriculares prematuros (también conocidos como contracciones ventriculares prematuras o PVC). Estos latidos se originan en los ventrículos y ocurren antes de lo esperado en un ciclo cardíaco normal, lo que puede ser indicativo de una arritmia ventricular.

    S: Latidos supraventriculares prematuros (también conocidos como contracciones auriculares prematuras o PAC). Estos latidos se originan en las aurículas y ocurren antes de lo esperado en un ciclo cardíaco normal, lo que puede ser indicativo de una arritmia supraventricular.

    F: Latidos de fusión. Estos latidos son el resultado de la fusión de un latido normal y un latido ectópico (originado fuera del nódulo sinusal), lo que produce un latido con características de ambos.

    Q: Latidos no clasificables o de mala calidad. Estos latidos no se pueden clasificar con precisión en ninguna de las categorías anteriores, ya sea debido a artefactos en la señal, ruido o características ambiguas.



## **Conclusiones**

    - El modelo de clasificación desarrollado, muestra buena eficacia en la clasificación para los segmentos que se le suministran en formato CSV. 
    
    - Para los segmentos que se extraen de la imagen, el proceso de clasificación tiene un buen desempeño, considerando que el vector se reconstruyó a partir de una imagen binarizada y es una representación aproximada de la real. 
    
    - Se requiere fortalecer el procesamiento de imagen, ya que cuando la imagen tiene imperfecciones como papel arrugado, se dificulta la correcta extracción del vector.
    
    - La capacidad de generalizar del modelo se puso a prueba con una nueva base de datos, obteniendo excelentes resultados en la clasificación.
    
    - Este ejercicio desarrollado de clasificación, sienta las bases para complementar con nuevos modelos de ML que se enfoquen a temas más avanzados como lo es el diagnóstico médico. Aquí los modelos deberán integrar nuevas características como la edad, sexo, antecedentes y otros datos relevantes del paciente, que permitan predecir un diagnóstico en materia de cardiología como lo hace hoy día el software DXL de Philips.
